
# Predict Borrower Worthiness with Machine Learning

Can we build a machine learning model that can accurately predict if a borrower will pay off its loan on time?

To answer this question, we are going to be using a data set provided by the lendingclub https://www.lendingclub.com/. The Lendingsclub is a match-making website for borrowers and lenders.

Lenders willingly provide loans to borrowers, nevertheless, there is always a risk that this latter won't be able to repay. Yet, the higher the risk, the higher the interest being paid, i.e. the money made for the lender.

Predicting credit-worthiness, will enable lenders to find the sweet spot between risk and return.

The dataset covers the Lending Club 2007-2011 Data - one can get the dataset on Kaggle via this link: https://www.kaggle.com/samaxtech/lending-club-20072011-data/data

In [39]:
import pandas as pd
import numpy as np

# Exploring the Data

In [40]:
loans = pd.read_csv('loans_2007.csv')

In [41]:
loans.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42538 entries, 0 to 42537
Data columns (total 52 columns):
id                            42538 non-null object
member_id                     42535 non-null float64
loan_amnt                     42535 non-null float64
funded_amnt                   42535 non-null float64
funded_amnt_inv               42535 non-null float64
term                          42535 non-null object
int_rate                      42535 non-null object
installment                   42535 non-null float64
grade                         42535 non-null object
sub_grade                     42535 non-null object
emp_title                     39909 non-null object
emp_length                    41423 non-null object
home_ownership                42535 non-null object
annual_inc                    42531 non-null float64
verification_status           42535 non-null object
issue_d                       42535 non-null object
loan_status                   42535 non-null object
p

columns description https://docs.google.com/spreadsheets/d/191B2yJ4H1ZPXq0_ByhUgWMFZOYem5jFz0Y3by_7YBY4/edit#gid=2081333097

In [42]:
loans.head()

,id,member_id,loan_amnt,funded_amnt,funded_amnt_inv,term,int_rate,installment,grade,sub_grade,...,last_pymnt_amnt,last_credit_pull_d,collections_12_mths_ex_med,policy_code,application_type,acc_now_delinq,chargeoff_within_12_mths,delinq_amnt,pub_rec_bankruptcies,tax_liens
0,1077501,1296599.0,5000.0,5000.0,4975.0,36 months,10.65%,162.87,B,B2,...,171.62,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
1,1077430,1314167.0,2500.0,2500.0,2500.0,60 months,15.27%,59.83,C,C4,...,119.66,Sep-2013,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
2,1077175,1313524.0,2400.0,2400.0,2400.0,36 months,15.96%,84.33,C,C5,...,649.91,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
3,1076863,1277178.0,10000.0,10000.0,10000.0,36 months,13.49%,339.31,C,C1,...,357.48,Apr-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0
4,1075358,1311748.0,3000.0,3000.0,3000.0,60 months,12.69%,67.79,B,B5,...,67.79,Jun-2016,0.0,1.0,INDIVIDUAL,0.0,0.0,0.0,0.0,0.0


In [43]:
loans.isna().sum()

id                               0
member_id                        3
loan_amnt                        3
funded_amnt                      3
funded_amnt_inv                  3
term                             3
int_rate                         3
installment                      3
grade                            3
sub_grade                        3
emp_title                     2629
emp_length                    1115
home_ownership                   3
annual_inc                       7
verification_status              3
issue_d                          3
loan_status                      3
pymnt_plan                       3
purpose                          3
title                           16
zip_code                         3
addr_state                       3
dti                              3
delinq_2yrs                     32
earliest_cr_line                32
inq_last_6mths                  32
open_acc                        32
pub_rec                         32
revol_bal           

# Narrowing the data-set - part.1

The Machine Learning algorithm needs to be fed with data that is relevant.
Therefore we will be removing columns that include data that isn't useful for this exercise

After analysing each column, we can conclude that the following features need to be removed:

- id: randomly generated field by Lending Club for unique identification purposes only
- member_id: also a randomly generated field by Lending Club for unique identification purposes only
- funded_amnt: leaks data from the future (after the loan is already started to be funded)
- funded_amnt_inv: also leaks data from the future (after the loan is already started to be funded)
- grade: contains redundant information as the interest rate column (int_rate)
- sub_grade: also contains redundant information as the interest rate column (int_rate)
- emp_title: requires other data and a lot of processing to potentially be useful
- issue_d: leaks data from the future (after the loan is already completely funded)

In [44]:
loans_2007 = loans.drop(['id', 'member_id', 'funded_amnt', 'funded_amnt_inv', 'grade', 'sub_grade', 'emp_title', 'issue_d'], axis = 1)

# Narrowing the data-set - part.2

Further analysing the data we found;

- zip_code: redundant with the addr_state column since only the first 3 digits of the 5 digit zip code are visible (which only can be used to identify the state the borrower lives in)
- out_prncp: leaks data from the future, (after the loan already started to be paid off)
- out_prncp_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt: also leaks data from the future, (after the loan already started to be paid off)
- total_pymnt_inv: also leaks data from the future, (after the loan already started to be paid off)
- total_rec_prncp: also leaks data from the future, (after the loan already started to be paid off)


In [45]:
loans_cleaned  = loans_2007.drop(['zip_code', 'out_prncp', 'out_prncp_inv', 'total_pymnt', 'total_pymnt_inv', 'total_rec_prncp'], axis = 1)

# Narrowing the data-set - part.3

Finally, again rechecking the data, we found;

- total_rec_int: leaks data from the future, (after the loan already started to be paid off),
- total_rec_late_fee: also leaks data from the future, (after the loan already started to be paid off),
- recoveries: also leaks data from the future, (after the loan already started to be paid off),
- collection_recovery_fee: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_d: also leaks data from the future, (after the loan already started to be paid off),
- last_pymnt_amnt: also leaks data from the future, (after the loan already started to be paid off).

In [46]:
loans_really_cleaned = loans_cleaned.drop(['total_rec_int', 'total_rec_late_fee', 'recoveries', 'collection_recovery_fee', 'last_pymnt_d', 'last_pymnt_amnt'], axis =1)

In [47]:
loans = loans_really_cleaned

# Finding a target column

Which column in the data-set best describes if the loan was paid or not?

- the 'loan_status' column is the only that actually indicates whether the loan was paid on time or not

Therefore we will be using it as our target column

In [48]:
loans['loan_status'].value_counts()

Fully Paid                                             33136
Charged Off                                             5634
Does not meet the credit policy. Status:Fully Paid      1988
Current                                                  961
Does not meet the credit policy. Status:Charged Off      761
Late (31-120 days)                                        24
In Grace Period                                           20
Late (16-30 days)                                          8
Default                                                    3
Name: loan_status, dtype: int64

Let's explore what each term actually means using a quick Google search;

- Fully Paid	33136	Loan has been fully paid off.
- Charged Off	5634	Loan for which there is no longer a reasonable expectation of further payments.
- Does not meet the credit policy. Status:Fully Paid	1988	While the loan was paid off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
- Does not meet the credit policy. Status:Charged Off	761	While the loan was charged off, the loan application today would no longer meet the credit policy and wouldn't be approved on to the marketplace.
- In Grace Period	20	The loan is past due but still in the grace period of 15 days.
- Late (16-30 days)	8	Loan hasn't been paid in 16 to 30 days (late on the current payment).
- Late (31-120 days)	24	Loan hasn't been paid in 31 to 120 days (late on the current payment).
- Current	961	Loan is up to date on current payments.
- Default	3	Loan is defaulted on and no payment has been made for more than 121 days.

There are two possible end outcome of a loan;

- Being paid-off
- Not being paid-off

The columns describing these status are;
- Fully Paid (paid-off)
- Charged off (not paid-off)

Though the default column ressembles charged-off it seems that investors still believe there is small chance of it being paid-off eventually

Let's remove all rows that don't contain values other than 'Fully Paid' or 'Charged Off' for our target column ('loan_status')

In [49]:
array = ['Fully Paid', 'Charged Off']
loans_2 = loans[loans['loan_status'].isin(array)]

In [50]:
len(loans)

42538

In [51]:
loans_2['loan_status'].value_counts()

Fully Paid     33136
Charged Off     5634
Name: loan_status, dtype: int64

In [52]:
loans_3 = loans_2.replace(['Fully Paid', 'Charged Off'], [1, 0])

In [53]:
loans_3['loan_status'].value_counts()

1    33136
0     5634
Name: loan_status, dtype: int64

In [54]:
len(loans_3.columns)


32

Let's look at any columns that has only one value (other than NaN)

In [55]:
columns = loans_3.columns
columns_to_drop = []

for cols in columns:
    cleaning_up = loans_3[cols].dropna().unique()
    if len(cleaning_up) == 1:
        columns_to_drop.append(cols)

In [56]:
loans_4 = loans_3.drop(columns_to_drop, axis = 1)

In [57]:
len(loans_4.columns)

23

# Dealing with missing data

ML models cannot work with missing data, therefore a strategy must be adopted to either remove or change the data


In [58]:
null_counts = loans_4.isnull().sum()
null_counts

loan_amnt                  0
term                       0
int_rate                   0
installment                0
emp_length              1036
home_ownership             0
annual_inc                 0
verification_status        0
loan_status                0
purpose                    0
title                     11
addr_state                 0
dti                        0
delinq_2yrs                0
earliest_cr_line           0
inq_last_6mths             0
open_acc                   0
pub_rec                    0
revol_bal                  0
revol_util                50
total_acc                  0
last_credit_pull_d         2
pub_rec_bankruptcies     697
dtype: int64

Checking the columns, we see that emp_length is actually the length of unemployement, which is an important metric for credit worthiness, therefore we will keep the data despite its large number of missing data

<br>

As for the other missing data, we will remove in its entirety the column 'pub_rec_bankruptcies' as well as all rows with missing values

In [59]:
loans_5 = loans_4.drop(['pub_rec_bankruptcies'], axis =1)
loans_5 = loans_5.dropna()

loans_5.dtypes.value_counts()

object     11
float64    10
int64       1
dtype: int64

# Numerical Columns

Scikit-learn models only make use of numerical columns, therefore text columns must be either ignored or turned into numerical

In [60]:
numerical_columns_df = loans_5.select_dtypes(include=['float'])
numerical_columns_df.columns

Index(['loan_amnt', 'installment', 'annual_inc', 'dti', 'delinq_2yrs',
       'inq_last_6mths', 'open_acc', 'pub_rec', 'revol_bal', 'total_acc'],
      dtype='object')

In [61]:
object_columns_df = loans_5.select_dtypes(include=['object'])
object_columns = object_columns_df.columns
object_columns

Index(['term', 'int_rate', 'emp_length', 'home_ownership',
       'verification_status', 'purpose', 'title', 'addr_state',
       'earliest_cr_line', 'revol_util', 'last_credit_pull_d'],
      dtype='object')

Let's look at each object column unique values

In [62]:
for columns in object_columns:
    print(loans_5[columns].value_counts())

 36 months    28234
 60 months     9441
Name: term, dtype: int64
 10.99%    906
 11.49%    770
  7.51%    756
 13.49%    747
  7.88%    701
  7.49%    629
  9.99%    573
  7.90%    552
 11.71%    546
  5.42%    524
 11.99%    478
 10.37%    453
 12.69%    441
  8.49%    425
  6.03%    413
 12.99%    404
 12.42%    397
 10.65%    393
  5.79%    390
 11.86%    383
  7.29%    379
  6.62%    376
  8.90%    371
  9.63%    368
 10.59%    351
 14.27%    336
  9.91%    331
  5.99%    329
 12.53%    327
  7.14%    326
          ... 
 15.07%      2
 14.62%      2
 14.07%      2
 15.01%      2
 22.94%      2
 14.25%      2
 17.03%      2
 21.82%      2
 15.38%      2
 17.15%      2
 16.71%      1
 22.64%      1
 17.54%      1
 17.44%      1
 18.36%      1
 24.59%      1
 16.96%      1
 18.72%      1
 16.33%      1
 21.48%      1
 17.46%      1
 20.52%      1
 16.20%      1
 17.34%      1
 10.64%      1
 13.84%      1
 16.01%      1
 24.40%      1
 16.15%      1
 14.67%      1
Name: int_rate, Leng

The home_ownership, verification_status, emp_length, term, and addr_state columns all contain multiple discrete values

We should clean the emp_length column and treat it as a numerical one since the values have ordering (2 years of employment is less than 8 years).

It seems like the purpose and title columns do contain overlapping information but we'll keep the purpose column since it contains a few discrete values. In addition, the title column has data quality issues since many of the values are repeated with slight modifications (e.g. Debt Consolidation and Debt Consolidation Loan and debt consolidation).

Let's remove last_credit_pull_d, addr_state, title, and earliest_cr_line for not adding value

In [63]:
loans_5 = loans_5.drop(['last_credit_pull_d', 'addr_state', 'title', 'earliest_cr_line'], axis=1)

Let's convert the 'int_rate' and 'revol_util' columns into ing

In [64]:
loans_5['int_rate'] = loans_5['int_rate'].str.rstrip('%').astype('float')

In [65]:
loans_5['revol_util'] = loans_5['revol_util'].str.rstrip('%').astype('float')

In [66]:
mapping_dict = {
    "emp_length": {
        "10+ years": 10,
        "9 years": 9,
        "8 years": 8,
        "7 years": 7,
        "6 years": 6,
        "5 years": 5,
        "4 years": 4,
        "3 years": 3,
        "2 years": 2,
        "1 year": 1,
        "< 1 year": 0,
        "n/a": 0
    }
}

In [67]:
loans_6 = loans_5.replace(mapping_dict)

let's now get dummies for the columns home_ownership, verification_status, purpose, and term

In [68]:
dummy_df = pd.get_dummies(loans_6[["term", "verification_status", "home_ownership", "purpose"]])

In [69]:
loans_6 = pd.concat([loans_6, dummy_df], axis=1)

In [70]:
loans_6 = loans_6.drop(["verification_status", "term"], axis =1)

In [71]:
loans_6 = loans_6.drop(["home_ownership", "purpose"], axis =1)

# Making Predictions

- We now have cleaned data to feed the algorithm = features
- Our goal is to predict the data located in the column loan_status = target

# Setting-up an error metric

An error metric will indicate whether the machine learning model performs well or not.
If it does, it should help us identify loans that get repaid.
<br>

We are looking at reducing the possibility of two events;

- False positives: when we think loans will be repaid but actually aren't
- False negatives: when we think loans won't be repaid but actually are

# False Negatives & Positives

We can establish these two status by looking at two columns;

- loan_status actual
- prediction

From there we can generate the following;

- loan_status actual 0 prediction 1 = False Positive
- loan_status actual 1 prediction 1 = True Positive
- loan_status actual 0 prediction 0 = True Negative
- loan_status actual 1 prediction 0 = False Negative

In [72]:
loans_6["emp_length"].value_counts()

10    8545
0     4513
2     4303
3     4022
4     3353
5     3202
1     3176
6     2177
7     1714
8     1442
9     1228
Name: emp_length, dtype: int64

In [73]:
loans_6["emp_length"] = loans_6["emp_length"].astype(int)

In [74]:
loans_6.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 37675 entries, 0 to 39785
Data columns (total 38 columns):
loan_amnt                              37675 non-null float64
int_rate                               37675 non-null float64
installment                            37675 non-null float64
emp_length                             37675 non-null int32
annual_inc                             37675 non-null float64
loan_status                            37675 non-null int64
dti                                    37675 non-null float64
delinq_2yrs                            37675 non-null float64
inq_last_6mths                         37675 non-null float64
open_acc                               37675 non-null float64
pub_rec                                37675 non-null float64
revol_bal                              37675 non-null float64
revol_util                             37675 non-null float64
total_acc                              37675 non-null float64
term_ 36 months            

In [75]:
loans_6["purpose_debt_consolidation"].value_counts()

0    19924
1    17751
Name: purpose_debt_consolidation, dtype: int64

# Logistic Regression

Most algorithms can't deal with non-numeric or missing values, which is why we had to do so much data cleaning.

In [76]:
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

In [77]:
features = loans_6.drop(["loan_status"], axis = 1)
target = loans_6["loan_status"]

In [78]:
fitting = lr.fit(features, target)

C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [79]:
np.unique(predictions, return_counts = True)

NameError: name 'predictions' is not defined

# First Model Results

The model is obviously overfitted with overwhelming good results, this comes from the fact that we used the same data to train and predict

<br>

To improve the results we should use k-fold cross validation with cross_val_predict() function from the sklearn.model_selection package

In [80]:
from sklearn.model_selection import cross_val_predict
lr_2 = LogisticRegression()

predictions_2 = cross_val_predict(lr_2, features, target, cv = 3)

C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [81]:
np.unique(predictions_2, return_counts = True)

(array([0, 1], dtype=int64), array([   59, 37616], dtype=int64))

In [91]:
# False positives.
fp_filter = (predictions_2 == 1) & (loans_6["loan_status"] == 0)
fp_1 = len(predictions_2[fp_filter])

# True positives.
tp_filter = (predictions_2 == 1) & (loans_6["loan_status"] == 1)
tp_1 = len(predictions_2[tp_filter])

# False negatives.
fn_filter = (predictions_2 == 0) & (loans_6["loan_status"] == 1)
fn_1 = len(predictions_2[fn_filter])

# True negatives
tn_filter = (predictions_2 == 0) & (loans_6["loan_status"] == 0)
tn_1 = len(predictions_2[tn_filter])

In [92]:
# Rates
tpr_1 = tp_1 / (tp_1 + fn_1)
fpr_1 = fp_1 / (fp_1 + tn_1)

print(tpr_1)
print(fpr_1)

0.9987920460880877
0.9962887363147152


# Penalising the classifier

Our last results show that the logistic regression model doesn't take into account the imbalance in the data (i.e. we have more than 6 times loans being paid-off on time than not).

Therefore, we need to add some weight to the model to make-up for this imbalance

In [93]:
lr_3 = LogisticRegression(class_weight = "balanced")

In [100]:
predictions_3 = cross_val_predict(lr_3, features, target)

C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:2053: FutureWarning: You should specify a value for 'cv' instead of relying on the default value. The default value will change from 3 to 5 in version 0.22.
  warnings.warn(CV_WARNING, FutureWarning)
C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to sile

In [101]:
# False positives.
fp_filter = (predictions_3 == 1) & (loans_6["loan_status"] == 0)
fp_3 = len(predictions_3[fp_filter])

# True positives.
tp_filter = (predictions_3 == 1) & (loans_6["loan_status"] == 1)
tp_3 = len(predictions_3[tp_filter])

# False negatives.
fn_filter = (predictions_3 == 0) & (loans_6["loan_status"] == 1)
fn_3 = len(predictions_3[fn_filter])

# True negatives
tn_filter = (predictions_3 == 0) & (loans_6["loan_status"] == 0)
tn_3 = len(predictions_3[tn_filter])

In [102]:
# Rates
tpr_3 = tp_3 / (tp_3 + fn_3)
fpr_3 = fp_3 / (fp_3 + tn_3)

print(tpr_3)
print(fpr_3)

0.6737285510747693
0.39320838745592873


# Results

- We significantly improved false positive rate in the last screen by balancing the classes, which reduced true positive rate
- However, we'd only ever decide to fund 66% of the total loans (true positive rate), so we'd immediately reject a good amount of loans.

# Manual Penalties

- To get even better accuracy we can implement manual penalties

In [103]:
penalty = {
    0: 10,
    1: 1
}

lr_4 = LogisticRegression(class_weight=penalty)
predictions_4 = cross_val_predict(lr_4, features, target, cv=3)


C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [104]:
# False positives.
fp_filter_4 = (predictions_4 == 1) & (loans_6["loan_status"] == 0)
fp_4 = len(predictions_4[fp_filter_4])

# True positives.
tp_filter_4 = (predictions_4 == 1) & (loans_6["loan_status"] == 1)
tp_4 = len(predictions_4[tp_filter_4])

# False negatives.
fn_filter_4 = (predictions_4 == 0) & (loans_6["loan_status"] == 1)
fn_4 = len(predictions_4[fn_filter_4])

# True negatives
tn_filter_4 = (predictions_4 == 0) & (loans_6["loan_status"] == 0)
tn_4 = len(predictions_4[tn_filter_4])

# Rates
tpr_4 = tp_4 / (tp_4 + fn_4)
fpr_4 = fp_4 / (fp_4 + tn_4)

print(tpr_4)
print(fpr_4)

0.25150219909558325
0.0946372239747634


Assigning manual penalties lowered the false positive rate to 9%, and thus lowered risk. This comes at the expense of true positive rate i.e. we might miss out on some trades

Training a random forest algorithm may enable us to get more accuracy due to columns that correlate nonlinearly with loan_status

# Random Forest

In [105]:
from sklearn.ensemble import RandomForestClassifier

In [106]:
rf = RandomForestClassifier(class_weight ='balanced', random_state = 1)

In [88]:
predictions_5 = cross_val_predict(rf, features, target, cv =3)

C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)
C:\Users\Gabriel_pirastru\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\ensemble\forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


In [89]:
# False positives.
fp_filter_5 = (predictions_5 == 1) & (loans_6["loan_status"] == 0)
fp_5 = len(predictions_5[fp_filter_5])

# True positives.
tp_filter_5 = (predictions_5 == 1) & (loans_6["loan_status"] == 1)
tp_5 = len(predictions_5[tp_filter_5])

# False negatives.
fn_filter_5 = (predictions_5 == 0) & (loans_6["loan_status"] == 1)
fn_5 = len(predictions_5[fn_filter_5])

# True negatives
tn_filter_5 = (predictions_5 == 0) & (loans_6["loan_status"] == 0)
tn_5 = len(predictions_5[tn_filter_5])

# Rates
tpr_5 = tp_5 / (tp_5 + fn_5)
fpr_5 = fp_5 / (fp_5 + tn_5)

print(tpr_5)
print(fpr_5)

0.9687170909991947
0.9200222675821117


Unfortunately, using a random forest classifier didn't improve our false positive rate. The model is likely weighting too heavily on the 1 class, and still mostly predicting 1s. We could fix this by applying a harsher penalty for misclassifications of 0s.

Ultimately, our best model had a false positive rate of nearly 9%, and a true positive rate of nearly 24%. For a conservative investor, this means that they make money as long as the interest rate is high enough to offset the losses from 9% of borrowers defaulting, and that the pool of 24% of borrowers is large enough to make enough interest money to offset the losses.



If we had randomly picked loans to fund, borrowers would have defaulted on 14.5% of them, and our model is better than that, although we're excluding more loans than a random strategy would. Given this, there's still quite a bit of room to improve:

- We can tweak the penalties further.
- We can try models other than a random forest and logistic regression.
- We can use some of the columns we discarded to generate better features.
- We can ensemble multiple models to get more accurate predictions.
- We can tune the parameters of the algorithm to achieve higher performance.